In [1]:
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import squidpy as sq
from sklearn.metrics.pairwise import cosine_similarity
import statistics
import pandas as pd
import json
import pickle
from anndata import AnnData
import pathlib
import skimage
import seaborn as sns
import tangram as tg
from scipy.spatial import distance

/home/apon/anaconda3/envs/tangram-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
adata = sc.read_h5ad("../lucas_data/real_st_spapros_merfish.h5ad")

In [4]:
x_coords = adata.obs['X'].values
y_coords = adata.obs['Y'].values

x_min, x_max = np.min(x_coords), np.max(x_coords)
y_min, y_max = np.min(y_coords), np.max(y_coords)
ratio = (x_max - x_min) / (y_max - y_min)

# Choose grid dimensions (193x3 or 3x193) based on aspect ratio
log_ratio = np.log(ratio)
if abs(log_ratio - np.log(193/3)) <= abs(log_ratio - np.log(3/193)):
    n_x, n_y = 193, 3  # Wide grid
else:
    n_x, n_y = 3, 193  # Tall grid

# Generate bin edges for the grid
x_bins = np.linspace(x_min, x_max, n_x + 1)
y_bins = np.linspace(y_min, y_max, n_y + 1)

# Assign cells to voxels using digitization
x_bin_indices = np.digitize(x_coords, x_bins) - 1
y_bin_indices = np.digitize(y_coords, y_bins) - 1

# Clip indices to handle edge cases (values at max bounds)
x_bin_indices = np.clip(x_bin_indices, 0, n_x - 1)
y_bin_indices = np.clip(y_bin_indices, 0, n_y - 1)

# Calculate unique voxel ID for each cell (0 to 578)
voxel_ids = y_bin_indices * n_x + x_bin_indices

# Add voxel assignments to the AnnData object
adata.obs['voxel'] = voxel_ids


In [9]:
adata.obs['voxel']

Cell
Cell_1         0
Cell_2         0
Cell_3         3
Cell_4         9
Cell_5         0
            ... 
Cell_5796    328
Cell_5797    314
Cell_5798    136
Cell_5799    575
Cell_5800    542
Name: voxel, Length: 5793, dtype: int64